In [1]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 95.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 129.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 65.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 71.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 104.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 124.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 135.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 138.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# %load_ext autoreload 
# %autoreload 2

from imports import *
import queue_system as qs
from queue_system import *

seed = random.seed(10)
seed_np = np.random.seed(10)

# Aggregation

In [3]:
with open('Q/Q_start.pkl', "rb") as fp:
    Q_start = pickle.load(fp)

In [4]:
weights_dict = {}
# model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max)
# model = StochasticMatching(graph, arrival_rates, queue_max)

len_updates = 100
res = int(len_updates / 50)      


weights_dict['-1'] = {state: np.array([1/K] * K) for state in state_space_ind.keys()}

repeat_weights = 10
learning_rate = 0.1
num_episodes = 10

In [5]:
%%time 
random.seed(6)
np.random.seed(6)
for n in range(repeat_weights):
    weights_dict = {}
    model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max, lambda_l2=0, lambda_l1=0, lr=1e-3, Q=Q_start)
    model = StochasticMatching(graph, arrival_rates, queue_max)
    weights_dict['-1'] =  np.ones([num_states, K]) / K
    
    lr = 2
    
    for i in tqdm(range(len_updates + 1)):
        if (i <= 200) and (i%10 == 0):
            weights = model_agg.train_actor_critic(num_episodes, lr)
            weights_dict[str(i)] = copy.deepcopy(weights) #.copy()
        else:
            weights = model_agg.train_actor_critic(num_episodes, lr)
            
        lr = lr * 0.9
            
    with open('weights/weights_actor_critic2_' + str(n) + '.pkl', 'wb') as output:
        pickle.dump(weights_dict, output) 
        
## stoch match 3 PROVA ENTRAMBI MSE

/home/chiara.mignacco/Aggreg-rl-code/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
  3%|▎         | 3/101 [05:07<2:47:37, 102.62s/it]


KeyboardInterrupt: 